In [1]:
#%%capture
#!wget "https://repo1.maven.org/maven2/org/apache/spark/spark-sql-kafka-0-10_2.12/3.5.0/spark-sql-kafka-0-10_2.12-3.5.0.jar"
#!wget "https://repo1.maven.org/maven2/org/apache/spark/spark-streaming-kafka-0-10_2.12/3.5.0/spark-streaming-kafka-0-10_2.12-3.5.0.jar"

--2025-11-18 07:39:49--  https://repo1.maven.org/maven2/org/apache/spark/spark-sql-kafka-0-10_2.12/3.5.0/spark-sql-kafka-0-10_2.12-3.5.0.jar
Resolving repo1.maven.org (repo1.maven.org)... 104.18.18.12, 104.18.19.12, 2606:4700::6812:120c, ...
Connecting to repo1.maven.org (repo1.maven.org)|104.18.18.12|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 432335 (422K) [application/java-archive]
Saving to: ‘spark-sql-kafka-0-10_2.12-3.5.0.jar’

spark-sql-kafka-0-1 100%[===================>] 422.20K  2.09MB/s    in 0.2s    

2025-11-18 07:39:49 (2.09 MB/s) - ‘spark-sql-kafka-0-10_2.12-3.5.0.jar’ saved [432335/432335]

--2025-11-18 07:39:50--  https://repo1.maven.org/maven2/org/apache/spark/spark-streaming-kafka-0-10_2.12/3.5.0/spark-streaming-kafka-0-10_2.12-3.5.0.jar
Resolving repo1.maven.org (repo1.maven.org)... 104.18.18.12, 104.18.19.12, 2606:4700::6812:120c, ...
Connecting to repo1.maven.org (repo1.maven.org)|104.18.18.12|:443... connected.
HTTP request sent, awa

In [2]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.5.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0 pyspark-shell'

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [4]:
spark = SparkSession.builder.appName("WordCount").getOrCreate()

In [5]:
lines = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "10.10.83.206:8097") \
  .option("subscribe", "input") \
  .option("startingOffsets", "latest") \
  .load()
#earliest

In [6]:
lines.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [12]:
#Lab 1.1
if False:
    query = lines \
      .selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)", "topic", "partition", "offset", "timestamp","timestampType") \
      .writeStream \
      .outputMode("update") \
      .format("console") \
      .option("truncate", False) \
      .start()
    
    query.awaitTermination()

# 1. complete
# Complete output mode not supported when there are no streaming aggregations on streaming DataFrames/Datasets;
# 2. append is fine.
# 3. update is fine.

In [19]:
words = lines.select(
   explode(
       split(lines.value, " ").alias("word")
   ).alias("word")
)

In [20]:
words.printSchema()

root
 |-- word: string (nullable = false)



In [21]:

# Generate running word count
wordCounts = words.groupBy("word").count()

In [35]:
wordCounts.printSchema()

root
 |-- word: string (nullable = false)
 |-- count: long (nullable = false)



In [30]:
#Lab 1.2
if False:
    # Start running the query that prints the running counts to the console
    query = wordCounts \
        .writeStream \
        .outputMode("update") \
        .format("console") \
        .option("truncate", False) \
        .start()
    
    query.awaitTermination()


In [40]:
#Lab 1.3
if False:
    query = wordCounts.selectExpr("word AS key", "CAST(count AS STRING) AS value") \
        .writeStream \
        .outputMode("complete") \
        .format("kafka") \
        .option("kafka.bootstrap.servers", "10.10.83.206:8097") \
        .option("topic", "output") \
        .option("checkpointLocation", "/tmp/chk1") \
        .start()
    
    query.awaitTermination()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [33]:
for q in spark.streams.active:
    q.stop()